# ui.R

In [ ]:
# Load required libraries
require(leaflet)
require(shiny)

# Create a Shiny UI
shinyUI(
  fluidPage(
    padding = 5,
    titlePanel(
      title = div(
        "Bike-sharing demand prediction app",
        tags$h3("by Your Name")  # Add your name here
      )
    ),
    # Create a side-bar layout
    sidebarLayout(
      # Create a main panel to show cities on a leaflet map
      mainPanel(
        leafletOutput("city_bike_map", height = 600)  # Adjusted height for better display
      ),
      # Create a side bar to show detailed plots for a city
      sidebarPanel(
        # select drop down list to select city (Task 2)
        selectInput("city_dropdown", "Select a city:",
                    choices = c("All", "New York", "Paris", "Suzhou", "London"),
                    selected = "All"),
        # Plot output for temperature trend (Task 1)
        plotOutput("temp_line", height = 300),  # Added plot output for temperature trend
        # Plot output for bike-sharing demand prediction trend (Task 2)
        plotOutput("bike_line", height = 300, click = "plot_click"),  # Added plot output for bike-sharing prediction trend
        # Text output for clicked point details
        verbatimTextOutput("bike_date_output"),  # Added verbatimTextOutput for clicked point details
        # Task 3: Plot output for humidity vs bike-sharing demand prediction correlation
        plotOutput("humidity_pred_chart", height = 300)  # Added plot output for humidity vs bike-sharing prediction correlation
      )
    ),
    hr(),
    footerPanel(
      tags$h5("Developed by Your Name")  # Add your name here
    )
  )
)


# server.R

In [ ]:
# Install and import required libraries
require(shiny)
require(ggplot2)
require(leaflet)
require(tidyverse)
require(httr)
require(scales)
# Import model_prediction R which contains methods to call OpenWeather API
# and make predictions
source("model_prediction.R")

# Function to generate test weather data
test_weather_data_generation <- function() {
  city_weather_bike_df <- generate_city_weather_bike_data()
  stopifnot(length(city_weather_bike_df) > 0)
  print(city_weather_bike_df)
  return(city_weather_bike_df)
}

# Create a RShiny server
shinyServer(function(input, output) {
  # Test generate_city_weather_bike_data() function
  city_weather_bike_df <- test_weather_data_generation()

  # Create another data frame called cities_max_bike with each row contains city location info and max bike
  # prediction for the city
  cities_max_bike <- city_weather_bike_df %>%
    group_by(CITY_ASCII) %>%
    summarise(MAX_BIKE_PREDICTION = max(BIKE_PREDICTION),
              LAT = first(LAT),
              LNG = first(LNG),
              LABEL = first(LABEL),
              DETAILED_LABEL = first(DETAILED_LABEL))

  # Define color factor
  color_levels <- colorFactor(c("green", "yellow", "red"), levels = c("small", "medium", "large"))

  # Render leaflet map based on dropdown selection
  observeEvent(input$city_dropdown, {
    if (input$city_dropdown == "All") {
      output$city_bike_map <- renderLeaflet({
        leaflet(data = cities_max_bike) %>%
          addTiles() %>%
          addCircleMarkers(
            radius = ~ifelse(MAX_BIKE_PREDICTION <= 50, 6,
                             ifelse(MAX_BIKE_PREDICTION <= 100, 10, 12)),
            color = ~color_levels(ifelse(MAX_BIKE_PREDICTION <= 50, "small",
                                         ifelse(MAX_BIKE_PREDICTION <= 100, "medium", "large"))),
            stroke = FALSE,
            fillOpacity = 0.5,
            label = ~as.character(LABEL),
            popup = ~as.character(LABEL)
          ) %>%
          setView(lng = cities_max_bike$LNG[1], lat = cities_max_bike$LAT[1], zoom = 4)
      })
    } else {
      selected_city <- cities_max_bike %>% filter(CITY_ASCII == input$city_dropdown)
      output$city_bike_map <- renderLeaflet({
        leaflet(data = selected_city) %>%
          addTiles() %>%
          addMarkers(lng = ~LNG, lat = ~LAT,
                     popup = ~as.character(DETAILED_LABEL))
      })
    }

    # Task 1: Render static temperature trend line
    output$temp_line <- renderPlot({
      if (input$city_dropdown != "All") {
        selected_city_data <- city_weather_bike_df %>%
          filter(CITY_ASCII == input$city_dropdown) %>%
          arrange(FORECASTDATETIME)  # Ensure data is sorted by FORECASTDATETIME

        ggplot(selected_city_data, aes(x = FORECASTDATETIME, y = TEMPERATURE)) +
          geom_line(color = "blue") +
          geom_point(color = "blue") +
          geom_text(aes(label = paste0(TEMPERATURE, "°C")), vjust = -0.5, hjust = -0.5) +
          labs(title = "Temperature Trend (Next 5 Days)",
               x = "Date", y = "Temperature (°C)")
      }
    })

    # Task 2: Render interactive bike-sharing demand prediction trend line
    output$bike_line <- renderPlot({
      if (input$city_dropdown != "All") {
        selected_city_data <- city_weather_bike_df %>%
          filter(CITY_ASCII == input$city_dropdown) %>%
          arrange(FORECASTDATETIME)  # Ensure data is sorted by FORECASTDATETIME

        p <- ggplot(selected_city_data, aes(x = FORECASTDATETIME, y = BIKE_PREDICTION)) +
          geom_line(color = "green") +
          geom_point(color = "green") +
          geom_text(aes(label = paste0(BIKE_PREDICTION)), vjust = -0.5, hjust = -0.5) +
          labs(title = "Bike-sharing Demand Prediction Trend",
               x = "Date", y = "Bike Prediction")

        if (!is.null(input$plot_click)) {
          click <- input$plot_click
          near_point <- nearest_data(selected_city_data, click)
          p <- p +
            geom_text(data = near_point, aes(label = paste0("Demand: ", BIKE_PREDICTION, "\nDate: ", FORECASTDATETIME)),
                      color = "black", vjust = -1, hjust = -0.5, size = 4, parse = TRUE)
        }

        p
      }
    })

    # Task 3: Render static humidity and bike-sharing demand prediction correlation plot
    output$humidity_pred_chart <- renderPlot({
      if (input$city_dropdown != "All") {
        selected_city_data <- city_weather_bike_df %>%
          filter(CITY_ASCII == input$city_dropdown)

        ggplot(selected_city_data, aes(x = HUMIDITY, y = BIKE_PREDICTION)) +
          geom_point(color = "blue", alpha = 0.5) +
          geom_smooth(method = "lm", formula = y ~ poly(x, 4), color = "red") +
          labs(title = "Humidity vs Bike-sharing Demand Prediction",
               x = "Humidity", y = "Bike Prediction")
      }
    })

    # Render text output for clicked point details
    output$bike_date_output <- renderText({
      if (!is.null(input$plot_click)) {
        click <- input$plot_click
        near_point <- nearest_data(selected_city_data, click)
        paste("Clicked Demand:", near_point$BIKE_PREDICTION, "\nClicked Date:", near_point$FORECASTDATETIME)
      }
    })
  })
})


model_predictor.R

In [ ]:
l['HUMIDITY'] +
    WIND_SPEED*model['WIND_SPEED'] + VISIBILITY*model['VISIBILITY']
  season_terms <- c()
  hour_terms <- c()
  # Calculate season related regression terms
  for(season in SEASONS) {
    season_term <- switch(season, 'SPRING'=model['SPRING'],'SUMMER'=model['SUMMER'],
                          'AUTUMN'=model['AUTUMN'], 'WINTER'=model['WINTER'])
    season_terms <- c(season_terms, season_term)
  }
  # Calculate hour related regression terms
  for(hour in HOURS){
    hour_term<- switch(as.character(hour),'0'=model['0'],'1'=model['1'],'2'=model['2'],'3'=model['3'],
                       '4'=model['4'],'5'=model['5'],'6'=model['6'],'7'=model['7'],
                       '8'=model['8'],'9'=model['9'],'10'=model['10'],'11'=model['11'],
                       '12'=model['12'],'13'=model['13'],'14'=model['14'],'15'=model['15'],'16'=model['16'],
                       '17'=model['17'],'18'=model['18'],'19'=model['19'],'20'=model['20'],
                       '21'=model['21'],'22'=model['22'],'23'=model['23'])
    hour_terms <- c(hour_terms, hour_term)

  }

regression_terms<-as.integer(weather_terms + season_terms + hour_terms)
for (i in 1:length(regression_terms))
  if(regression_terms[i] < 0)
  {
    regression_terms[i]<-0

  }
  else
  {
    regression_terms[i]<-regression_terms[i]
  }
return(regression_terms)
}


# Define a bike-sharing demand level, used for leaflet visualization
calculate_bike_prediction_level<- function(predictions) {
  levels <- c()
  for(prediction in predictions){
    if(prediction <= 1000 && prediction >= 0)
      levels <- c(levels, 'small')
    else if (prediction > 1000 && prediction < 3000)
      levels <- c(levels, 'medium')
    else
      levels <- c(levels, 'large')
  }
  return(levels)
}

# Generate a data frame containing weather forecasting and bike prediction data
# Generate a data frame containg weather forecasting and bike prediction data
generate_city_weather_bike_data <- function (){
  cities_df <- read_csv("selected_cities.csv")
  weather_df <- get_weather_forecaset_by_cities(cities_df$CITY_ASCII)
  results <- weather_df %>%
    mutate(BIKE_PREDICTION=predict_bike_demand(TEMPERATURE, HUMIDITY, WIND_SPEED, VISIBILITY, SEASONS, HOURS)) %>%
    mutate(BIKE_PREDICTION_LEVEL=calculate_bike_prediction_level(BIKE_PREDICTION))

  cities_bike_pred <- cities_df %>% left_join(results) %>%
    select(CITY_ASCII, LNG, LAT, TEMPERATURE, HUMIDITY, BIKE_PREDICTION, BIKE_PREDICTION_LEVEL, LABEL, DETAILED_LABEL, FORECASTDATETIME)
  return(cities_bike_pred)
}